In [124]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score

In [2]:
file = 'default_plus_chromatic_features_1059_tracks.txt'
dat = pd.read_csv(file, header=None)
X = dat.iloc[:,:-2]
y, orig = pd.factorize(dat.iloc[:,-1])

In [5]:
# Split the data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


In [14]:
# Construct the empricial covariance matrix

Xcov = (1/(Xmat_std.shape[0]-1))*np.dot(X_train.T, X_train)

In [15]:
Xcov.shape

(116, 116)

### Power Iteratoin and the first PC

In [96]:
# Power Iteration 
def power_iteration(mat, max_iter = 100):
        v = np.ones(mat.shape[0])
        
        for i in range(max_iter):
            z = mat @ v
            v = z/np.linalg.norm(z)
        
        lambda_ = v.T@mat@v
        
        return lambda_, v

In [97]:
lam, v = power_iteration(Xcov)

In [99]:
pca = PCA(n_components=1).fit(X_train) # get the top PCA component 
np.linalg.norm(pca.components_ - v)

2.7056085028033464e-12

In [101]:
pca.explained_variance_[0], v.T@Xcov@v

(43.71425766632031, 43.7142576663203)

The difference between output v from the power iteration and the top compoenent of PCA is nearly zero. Egiven values are about the same also.  

### Deflated covatiance matrix and Second PC

In [102]:
# Compute the delafted covariance matrix 

Xcov_deflate = Xcov - lam*np.outer(v,v.T)

In [103]:
lam2, v2  = power_iteration(Xcov_deflate)

In [104]:
pca2 = PCA(n_components=2).fit(X_train) # get the top PCA component 
np.linalg.norm(pca2.components_[1] - v2)

5.865592921007979e-09

In [106]:
pca2.explained_variance_[1], lam2

(13.017911061954894, 13.01791106195488)

### Propose for computing the $k^{th}$ largest eigenvector of X

In [110]:
def power_iteration_k(mat, k, max_iter = 100):
    for order in range(k):
        
        v = np.ones(mat.shape[0])
        
        for i in range(max_iter):
            z = mat @ v
            v = z/np.linalg.norm(z)
        
        lambda_ = v.T@mat@v
        
        mat = mat -  lambda_*np.outer(v,v.T)
    
    return lambda_, v

In [113]:
# Confirm for the 3rd PC
lam3, v = power_iteration_k(Xcov,3)

In [114]:
pca3 = PCA(n_components=3).fit(X_train) 
pca3.explained_variance_[2], lam3

(8.453858230413882, 8.453858230418561)

### Select the top 20 principal compnents

In [116]:
X_train_proj = PCA(n_components=20).fit_transform(X_train) 

In [120]:
knn_proj = NearestNeighbors(n_neighbors=1).fit(X_train_proj)
knn= NearestNeighbors(n_neighbors=1).fit(X_train)

In [131]:
knn_proj = KNeighborsClassifier(n_neighbors=1).fit(X_train_proj,y_train)
knn= KNeighborsClassifier(n_neighbors=1).fit(X_train,y_train)

In [132]:
X_test_proj = PCA(n_components=20).fit(X_train).transform(X_test)

In [133]:
proj_acc = accuracy_score(y_test, knn_proj.predict(X_test_proj))
acc = accuracy_score(y_test, knn.predict(X_test))

In [134]:
proj_acc, acc

(0.36792452830188677, 0.39622641509433965)

The accuracy score is higher for unproject data

### Bonus

In [165]:
# New stopping criterion: if the lambda stops changing (change less than a threshold), stop the iteration 

def power_iteration_new(mat, min_change = 1e-15):
        v = np.ones(mat.shape[0])
        lam = [0,1]
        
        while lam[-1]-lam[-2] > min_change:
            z = mat @ v
            v = z/np.linalg.norm(z)
        
            lam.append(v.T@mat@v)
        
        return lam[-1], v

In [166]:
lam, v = power_iteration_new(Xcov)

In [167]:
pca = PCA(n_components=1).fit(X_train) # get the top PCA component 
np.linalg.norm(pca.components_ - v)

2.3348809330661454e-09

In [169]:
lam,pca.explained_variance_[0]

(43.71425766632031, 43.714257666320265)

Given minimum changing threshold of 1e-5, the new algoritm performs the same as the old power iteration. 